# 00 - Clean files

In [1]:
!pip install pandas pyarrow liac-arff fastparquet


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


## Hurtlex

In [2]:
import os

import pandas as pd

# Configurações iniciais
lexic_file = './data/01-raw/HurtLex-PT/hurtlex_PT.tsv'

# Criar o DataFrame Pandas
raw_df = pd.read_csv(lexic_file, sep='\t')

# Separar os conservatives
conservative_df = raw_df[raw_df['level'] == 'conservative']

# Salvar como csv
new_path = './data/02-cleaned/'
os.makedirs(os.path.dirname(new_path), exist_ok=True)

file_name = 'hurtlex_PT_conservatives.csv'
conservative_df.to_csv(new_path + file_name, index=False)

## Datasets

In [3]:
import arff  # Necessário para ler o arquivo .arff (OffComBR)
import csv

# Configurações iniciais
datasets_dir = './data/01-raw/Datasets/'

datasets = {
		'Fortuna'     :['Fortuna/2019-05-28_portuguese_hate_speech_binary_classification.csv'],
		'HateBRXplain':['HateBRXplain/dataset/HateBRXplain/HateBRXplain.csv'],
		'OffComBR'    :['OffComBR/OffComBR3.arff'],
		'OLID-BR'     :[
				'OLID-BR/test.csv',
				'OLID-BR/train.csv'
				],
		'ToLD-BR'     :['ToLD-BR/ToLD-BR.csv'],
		'TuPy'        :['TuPy/binary/tupy_binary_vote.csv']
		}

# Dicionário para guardar o resultado final: {'NomeDataset': DataFrame}
dfs_dict = {}

for nome_dataset, lista_arquivos in datasets.items( ):
	lista_temp_dfs = []  # Lista temporária para casos com múltiplos arquivos (OLID-BR)

	for arquivo in lista_arquivos:
		# Cria o caminho completo
		caminho_completo = os.path.join(datasets_dir, arquivo)

		# Verifica a extensão e usa o leitor apropriado
		if arquivo.endswith('.csv'):
			# Atenção: alguns CSVs podem precisar de sep=';' ou encoding='latin1'
			df = pd.read_csv(caminho_completo)

		elif arquivo.endswith('.parquet'):
			# Adicionado engine='fastparquet' para evitar erro do PyArrow/Python 3.13
			df = pd.read_parquet(caminho_completo, engine='fastparquet')

		elif arquivo.endswith('.arff'):
			with open(caminho_completo, 'r') as f:
				# Carrega o ARFF como um dicionário
				dataset_arff = arff.load(f)

				# Extrai dados e nomes das colunas
				dados = dataset_arff['data']
				colunas = [atributo[0] for atributo in dataset_arff['attributes']]

				df = pd.DataFrame(dados, columns=colunas)

		lista_temp_dfs.append(df)

	# Concatena as partes (para o OLID-BR) e salva no dicionário principal
	if lista_temp_dfs:
		dfs_dict[nome_dataset] = pd.concat(lista_temp_dfs, ignore_index=True)
		print(f"Dataset '{nome_dataset}' carregado com sucesso. Shape: {dfs_dict[nome_dataset].shape}")

Dataset 'Fortuna' carregado com sucesso. Shape: (5670, 8)
Dataset 'HateBRXplain' carregado com sucesso. Shape: (7000, 6)
Dataset 'OffComBR' carregado com sucesso. Shape: (1033, 2)
Dataset 'OLID-BR' carregado com sucesso. Shape: (6952, 17)
Dataset 'ToLD-BR' carregado com sucesso. Shape: (21000, 7)
Dataset 'TuPy' carregado com sucesso. Shape: (10000, 2)


### Fortuna

In [4]:
print('##### LIMPEZA DE DADOS: Fortuna (binary version)')

# Pega o DataFrame
fortuna_df = dfs_dict['Fortuna'].copy( )
print(f"# Número de registros (raw): {fortuna_df.shape[0]}")

# Pega as colunas desejadas
columns_to_keep = ['text', 'hatespeech_comb']
fortuna_df = fortuna_df[columns_to_keep]
print(f"#\n# --- Colunas selecionadas: {columns_to_keep}")

# Padroniza os nomes das colunas
fortuna_df.columns = ['text', 'is_toxic']
print(f"# --- Colunas renomeadas: {fortuna_df.columns}")

# Normalizar a coluna 'is_toxic'
fortuna_df['is_toxic'] = fortuna_df['is_toxic'].map({'1':True, '0':False})
print(f"# --- Coluna 'is_toxic' normalizada para False (0) ou True (1)")

# Retira menções ao longo do texto, se antes do @ não for uma letra/palavra, e substitui por @user
fortuna_df['text'] = fortuna_df['text'].str.replace(r'(?<!\w)@\w+', '@user', regex=True)
fortuna_df['text'] = fortuna_df['text'].str.replace(r'\bUSER\b', '@user', regex=True)

print(f"# --- Menções normalizadas para '@user'")

# Retira URLs ao longo do texto
fortuna_df['text'] = fortuna_df['text'].str.replace(r'(?:https?://|www\.)\S+', '<url>', regex=True)
fortuna_df['text'] = fortuna_df['text'].str.replace(r'(?i)\blink\b(?=(?:\s*link\b)*\s*$)', '<url>', regex=True)
fortuna_df['text'] = fortuna_df['text'].str.replace(r'\bURL\b', '<url>', regex=True)
print(f"# --- Links normalizados para '<url>'")

# Normaliza os retweets, transformando para <RT>
fortuna_df['text'] = fortuna_df['text'].str.replace(r'\bRT\b', '<RT>', regex=True)
fortuna_df['text'] = fortuna_df['text'].str.replace(r'\brt\b', '<RT>', regex=True)
print(f"# --- Retweets normalizados para '<RT>'")

# Retira '\n' e substitui por ' '
fortuna_df['text'] = fortuna_df['text'].str.replace('\n', ' ', regex=False)
print(f"# --- \'\\n\' normalizado para ' '")

# Desconsidera todos os is_toxic sem valor (None)
fortuna_df = fortuna_df[fortuna_df['is_toxic'].notna( )]
print(f"#\n# --- Registros com is_toxic=None removidos")
print(f"# Quantidade de registros após remoção de valores incompletos: {fortuna_df.shape[0]}")

# Desconsidera entradas duplicadas
fortuna_df = fortuna_df.drop_duplicates(subset='text', keep='first')
print(f"#\n# --- Registros duplicados removidos")
print(f"# Quantidade de registros após remoção de duplicatas: {fortuna_df.shape[0]}")

# Salva o DataFrame final como CSV
fortuna_output = './data/02-cleaned/fortuna.csv'
fortuna_df.to_csv(fortuna_output, index=False, quoting=csv.QUOTE_NONNUMERIC)
print(f"#\n# Salvo em: {fortuna_output}")

##### LIMPEZA DE DADOS: Fortuna (binary version)
# Número de registros (raw): 5670
#
# --- Colunas selecionadas: ['text', 'hatespeech_comb']
# --- Colunas renomeadas: Index(['text', 'is_toxic'], dtype='object')
# --- Coluna 'is_toxic' normalizada para False (0) ou True (1)
# --- Menções normalizadas para '@user'
# --- Links normalizados para '<url>'
# --- Retweets normalizados para '<RT>'
# --- '\n' normalizado para ' '
#
# --- Registros com is_toxic=None removidos
# Quantidade de registros após remoção de valores incompletos: 0
#
# --- Registros duplicados removidos
# Quantidade de registros após remoção de duplicatas: 0
#
# Salvo em: ./data/02-cleaned/fortuna.csv


### HateBRXplain

In [5]:
print('##### LIMPEZA DE DADOS: HateBRXplain')

# Pega o DataFrame
hatebr_df = dfs_dict['HateBRXplain'].copy( )
print(f"# Número de registros (raw): {hatebr_df.shape[0]}")

# Pega as colunas desejadas
columns_to_keep = ['comment', 'offensive_label']
hatebr_df = hatebr_df[columns_to_keep]
print(f"#\n# --- Colunas selecionadas: {columns_to_keep}")

# Padroniza os nomes das colunas
hatebr_df.columns = ['text', 'is_toxic']
print(f"# --- Colunas renomeadas: {hatebr_df.columns}")

# Normalizar a coluna 'is_toxic'
hatebr_df['is_toxic'] = hatebr_df['is_toxic'].map({'1':True, '0':False})
print(f"# --- Coluna 'is_toxic' normalizada para False (0) ou True (1)")

# Retira menções ao longo do texto, se antes do @ não for uma letra/palavra, e substitui por @user
hatebr_df['text'] = hatebr_df['text'].str.replace(r'(?<!\w)@\w+', '@user', regex=True)
hatebr_df['text'] = hatebr_df['text'].str.replace(r'\bUSER\b', '@user', regex=True)
print(f"# --- Menções normalizadas para '@user'")

# Retira URLs ao longo do texto
hatebr_df['text'] = hatebr_df['text'].str.replace(r'(?:https?://|www\.)\S+', '<url>', regex=True)
hatebr_df['text'] = hatebr_df['text'].str.replace(r'(?i)\blink\b(?=(?:\s*link\b)*\s*$)', '<url>', regex=True)
hatebr_df['text'] = hatebr_df['text'].str.replace(r'\bURL\b', '<url>', regex=True)
print(f"# --- Links normalizados para '<url>'")

# Normaliza os retweets, transformando para <RT>
hatebr_df['text'] = hatebr_df['text'].str.replace(r'\bRT\b', '<RT>', regex=True)
hatebr_df['text'] = hatebr_df['text'].str.replace(r'\brt\b', '<RT>', regex=True)
print(f"# --- Retweets normalizados para '<RT>'")

# Retira '\n' e substitui por ' '
hatebr_df['text'] = hatebr_df['text'].str.replace('\n', ' ', regex=False)
print(f"# --- \'\\n\' normalizado para ' '")

# Desconsidera todos os is_toxic sem valor (None)
hatebr_df = hatebr_df[hatebr_df['is_toxic'].notna( )]
print(f"#\n# --- Registros com is_toxic=None removidos")
print(f"# Quantidade de registros após remoção de valores incompletos: {hatebr_df.shape[0]}")

# Desconsidera entradas duplicadas
hatebr_df = hatebr_df.drop_duplicates(subset='text', keep='first')
print(f"#\n# --- Registros duplicados removidos")
print(f"# Quantidade de registros após remoção de duplicatas: {hatebr_df.shape[0]}")

# Salva o DataFrame final como CSV
hatebr_output = './data/02-cleaned/hatebrxplain.csv'
hatebr_df.to_csv(hatebr_output, index=False, quoting=csv.QUOTE_NONNUMERIC)
print(f"#\n# Salvo em: {hatebr_output}")

# hatebr_df.head(100)

##### LIMPEZA DE DADOS: HateBRXplain
# Número de registros (raw): 7000
#
# --- Colunas selecionadas: ['comment', 'offensive_label']
# --- Colunas renomeadas: Index(['text', 'is_toxic'], dtype='object')
# --- Coluna 'is_toxic' normalizada para False (0) ou True (1)
# --- Menções normalizadas para '@user'
# --- Links normalizados para '<url>'
# --- Retweets normalizados para '<RT>'
# --- '\n' normalizado para ' '
#
# --- Registros com is_toxic=None removidos
# Quantidade de registros após remoção de valores incompletos: 0
#
# --- Registros duplicados removidos
# Quantidade de registros após remoção de duplicatas: 0
#
# Salvo em: ./data/02-cleaned/hatebrxplain.csv


### OffComBR3

In [6]:
print('##### LIMPEZA DE DADOS: OffComBR3')

# Pega o DataFrame
offcombr_df = dfs_dict['OffComBR'].copy( )
print(f"# Número de registros (raw): {offcombr_df.shape[0]}")

# Pega as colunas desejadas
columns_to_keep = ['document', '@@class']
offcombr_df = offcombr_df[columns_to_keep]
print(f"#\n# --- Colunas selecionadas: {columns_to_keep}")

# Padroniza os nomes das colunas
offcombr_df.columns = ['text', 'is_toxic']
print(f"# --- Colunas renomeadas: {offcombr_df.columns}")

# Normalizar a coluna 'is_toxic'
offcombr_df['is_toxic'] = offcombr_df['is_toxic'].map({'yes':True, 'no':False})
print(f"# --- Coluna 'is_toxic' normalizada para False (no) ou True (yes)")

# Retira menções ao longo do texto, se antes do @ não for uma letra/palavra, e substitui por @user
offcombr_df['text'] = offcombr_df['text'].str.replace(r'(?<!\w)@\w+', '@user', regex=True)
offcombr_df['text'] = offcombr_df['text'].str.replace(r'\bUSER\b', '@user', regex=True)

print(f"# --- Menções normalizadas para '@user'")

# Retira URLs ao longo do texto
offcombr_df['text'] = offcombr_df['text'].str.replace(r'(?:https?://|www\.)\S+', '<url>', regex=True)
offcombr_df['text'] = offcombr_df['text'].str.replace(r'(?i)\blink\b(?=(?:\s*link\b)*\s*$)', '<url>', regex=True)
offcombr_df['text'] = offcombr_df['text'].str.replace(r'\bURL\b', '<url>', regex=True)
print(f"# --- Links normalizados para '<url>'")

# Normaliza os retweets, transformando para <RT>
offcombr_df['text'] = offcombr_df['text'].str.replace(r'\bRT\b', '<RT>', regex=True)
offcombr_df['text'] = offcombr_df['text'].str.replace(r'\brt\b', '<RT>', regex=True)
print(f"# --- Retweets normalizados para '<RT>'")

# Retira '\n' e substitui por ' '
offcombr_df['text'] = offcombr_df['text'].str.replace('\n', ' ', regex=False)
print(f"# --- \'\\n\' normalizado para ' '")

# Desconsidera todos os is_toxic sem valor (None)
offcombr_df = offcombr_df[offcombr_df['is_toxic'].notna( )]
print(f"#\n# --- Registros com is_toxic=None removidos")
print(f"# Quantidade de registros após remoção de valores incompletos: {offcombr_df.shape[0]}")

# Desconsidera entradas duplicadas
offcombr_df = offcombr_df.drop_duplicates(subset='text', keep='first')
print(f"#\n# --- Registros duplicados removidos")
print(f"# Quantidade de registros após remoção de duplicatas: {offcombr_df.shape[0]}")

# Salva o DataFrame final como CSV
offcombr_output = './data/02-cleaned/offcombr3.csv'
offcombr_df.to_csv(offcombr_output, index=False, quoting=csv.QUOTE_NONNUMERIC)
print(f"#\n# Salvo em: {offcombr_output}")

# offcombr_df.head(100)

##### LIMPEZA DE DADOS: OffComBR3
# Número de registros (raw): 1033
#
# --- Colunas selecionadas: ['document', '@@class']
# --- Colunas renomeadas: Index(['text', 'is_toxic'], dtype='object')
# --- Coluna 'is_toxic' normalizada para False (no) ou True (yes)
# --- Menções normalizadas para '@user'
# --- Links normalizados para '<url>'
# --- Retweets normalizados para '<RT>'
# --- '\n' normalizado para ' '
#
# --- Registros com is_toxic=None removidos
# Quantidade de registros após remoção de valores incompletos: 1033
#
# --- Registros duplicados removidos
# Quantidade de registros após remoção de duplicatas: 1029
#
# Salvo em: ./data/02-cleaned/offcombr3.csv


### OLID

In [7]:
print('##### LIMPEZA DE DADOS: OLID-BR (treino e teste)')

# Pega o DataFrame
olidbr_df = dfs_dict['OLID-BR'].copy( )
print(f"# Número de registros (raw): {olidbr_df.shape[0]}")

# Pega as colunas desejadas
columns_to_keep = ['text', 'is_offensive']
olidbr_df = olidbr_df[columns_to_keep]
print(f"#\n# --- Colunas selecionadas: {columns_to_keep}")

# Padroniza os nomes das colunas
olidbr_df.columns = ['text', 'is_toxic']
print(f"# --- Colunas renomeadas: {olidbr_df.columns}")

# Normalizar a coluna 'is_toxic'
olidbr_df['is_toxic'] = olidbr_df['is_toxic'].map({'NOT':False, 'OFF':True})
print(f"# --- Coluna 'is_toxic' normalizada para False (NOT) ou True (OFF)")

# Retira menções ao longo do texto, se antes do @ não for uma letra/palavra, e substitui por @user
# No caso desse dataset, já normalizado, faz o mesmo com 'USER', para padronizar com os outros datasets
olidbr_df['text'] = olidbr_df['text'].str.replace(r'(?<!\w)@\w+', '@user', regex=True)
olidbr_df['text'] = olidbr_df['text'].str.replace(r'\bUSER\b', '@user', regex=True)
print(f"# --- Menções normalizadas para '@user'")

# Retira URLs ao longo do texto
# No caso desse dataset, já normalizado, faz o mesmo com 'URL', para padronizar com os outros datasets
olidbr_df['text'] = olidbr_df['text'].str.replace(r'(?:https?://|www\.)\S+', '<url>', regex=True)
olidbr_df['text'] = olidbr_df['text'].str.replace(r'\bURL\b', '<url>', regex=True)
olidbr_df['text'] = olidbr_df['text'].str.replace(r'(?i)\blink\b(?=(?:\s*link\b)*\s*$)', '<url>', regex=True)
print(f"# --- Links normalizados para '<url>'")

# Normaliza os retweets, transformando para <RT>
olidbr_df['text'] = olidbr_df['text'].str.replace(r'\bRT\b', '<RT>', regex=True)
olidbr_df['text'] = olidbr_df['text'].str.replace(r'\brt\b', '<RT>', regex=True)
print(f"# --- Retweets normalizados para '<RT>'")

# Retira '\n' e substitui por ' '
olidbr_df['text'] = olidbr_df['text'].str.replace('\n', ' ', regex=False)
print(f"# --- \'\\n\' normalizado para ' '")

# Desconsidera todos os is_toxic sem valor (None)
olidbr_df = olidbr_df[olidbr_df['is_toxic'].notna( )]
print(f"#\n# --- Registros com is_toxic=None removidos")
print(f"# Quantidade de registros após remoção de valores incompletos: {olidbr_df.shape[0]}")

# Desconsidera entradas duplicadas
olidbr_df = olidbr_df.drop_duplicates(subset='text', keep='first')
print(f"#\n# --- Registros duplicados removidos")
print(f"# Quantidade de registros após remoção de duplicatas: {olidbr_df.shape[0]}")

# Salva o DataFrame final como CSV
olidbr_output = './data/02-cleaned/olidbr.csv'
olidbr_df.to_csv(olidbr_output, index=False, quoting=csv.QUOTE_NONNUMERIC)
print(f"#\n# Salvo em: {olidbr_output}")


##### LIMPEZA DE DADOS: OLID-BR (treino e teste)
# Número de registros (raw): 6952
#
# --- Colunas selecionadas: ['text', 'is_offensive']
# --- Colunas renomeadas: Index(['text', 'is_toxic'], dtype='object')
# --- Coluna 'is_toxic' normalizada para False (NOT) ou True (OFF)
# --- Menções normalizadas para '@user'
# --- Links normalizados para '<url>'
# --- Retweets normalizados para '<RT>'
# --- '\n' normalizado para ' '
#
# --- Registros com is_toxic=None removidos
# Quantidade de registros após remoção de valores incompletos: 6952
#
# --- Registros duplicados removidos
# Quantidade de registros após remoção de duplicatas: 6952
#
# Salvo em: ./data/02-cleaned/olidbr.csv


### ToLD-BR

In [8]:
print('##### LIMPEZA DE DADOS: ToLD-BR')

# Pega o DataFrame
toldbr_df = dfs_dict['ToLD-BR'].copy( )
print(f"# Número de registros (raw): {toldbr_df.shape[0]}")

# Pega as colunas desejadas
# No caso desse dataset, precisamos identificar quais as mensagens que são toxicas -- quando tiver pelo menos um 1 nas colunas de tipo de ofensa/odio
cols_toxicas = ['homophobia', 'obscene', 'insult', 'racism', 'misogyny', 'xenophobia']
toldbr_df['is_toxic'] = toldbr_df[cols_toxicas].max(axis=1)
print(f"#\n# --- Coluna criada: \'is_toxic\'")

toldbr_df = toldbr_df[['text', 'is_toxic']]

# Padroniza os nomes das colunas
# toldbr_df.columns = ['text', 'is_toxic']
print(f"# --- Colunas já nomeadas: {toldbr_df.columns}")

# Normalizar a coluna 'is_toxic'
toldbr_df['is_toxic'] = toldbr_df['is_toxic'].map({'1.0':True, '0.0':False})
print(f"# --- Coluna 'is_toxic' normalizada para False (0) ou True (1)")

# Retira menções ao longo do texto, se antes do @ não for uma letra/palavra, e substitui por @user
toldbr_df['text'] = toldbr_df['text'].str.replace(r'(?<!\w)@\w+', '@user', regex=True)
toldbr_df['text'] = toldbr_df['text'].str.replace(r'\bUSER\b', '@user', regex=True)

print(f"# --- Menções normalizadas para '@user'")

# Retira URLs ao longo do texto
# No caso esse foi anonimizado tb. Utilizaremos <url> quando link estiver por ultimo
toldbr_df['text'] = toldbr_df['text'].str.replace(r'(?:https?://|www\.)\S+', '<url>', regex=True)
toldbr_df['text'] = toldbr_df['text'].str.replace(r'(?i)\blink\b(?=(?:\s*link\b)*\s*$)', '<url>', regex=True)
toldbr_df['text'] = toldbr_df['text'].str.replace(r'\bURL\b', '<url>', regex=True)

print(f"# --- Links normalizados para '<url>'")

# Normaliza os retweets, transformando para <RT>
toldbr_df['text'] = toldbr_df['text'].str.replace(r'\bRT\b', '<RT>', regex=True)
toldbr_df['text'] = toldbr_df['text'].str.replace(r'\brt\b', '<RT>', regex=True)
print(f"# --- Retweets normalizados para '<RT>'")


# Retira '\n' e substitui por ' '
toldbr_df['text'] = toldbr_df['text'].str.replace('\n', ' ', regex=False)
print(f"# --- \'\\n\' normalizado para ' '")

# Desconsidera todos os is_toxic sem valor (None)
toldbr_df = toldbr_df[toldbr_df['is_toxic'].notna( )]
print(f"#\n# --- Registros com is_toxic=None removidos")
print(f"# Quantidade de registros após remoção de valores incompletos: {toldbr_df.shape[0]}")

# Desconsidera entradas duplicadas
toldbr_df = toldbr_df.drop_duplicates(subset='text', keep='first')
print(f"#\n# --- Registros duplicados removidos")
print(f"# Quantidade de registros após remoção de duplicatas: {toldbr_df.shape[0]}")

# Salva o DataFrame final como CSV
toldbr_output = './data/02-cleaned/toldbr.csv'
toldbr_df.to_csv(toldbr_output, index=False, quoting=csv.QUOTE_NONNUMERIC)
print(f"#\n# Salvo em: {toldbr_output}")

##### LIMPEZA DE DADOS: ToLD-BR
# Número de registros (raw): 21000
#
# --- Coluna criada: 'is_toxic'
# --- Colunas já nomeadas: Index(['text', 'is_toxic'], dtype='object')
# --- Coluna 'is_toxic' normalizada para False (0) ou True (1)
# --- Menções normalizadas para '@user'
# --- Links normalizados para '<url>'
# --- Retweets normalizados para '<RT>'
# --- '\n' normalizado para ' '
#
# --- Registros com is_toxic=None removidos
# Quantidade de registros após remoção de valores incompletos: 0
#
# --- Registros duplicados removidos
# Quantidade de registros após remoção de duplicatas: 0
#
# Salvo em: ./data/02-cleaned/toldbr.csv


### Typy

In [9]:
print('##### LIMPEZA DE DADOS: TuPy (versão not-expanded)')

# Pega o DataFrame
tupy_df = dfs_dict['TuPy'].copy( )
print(f"# Número de registros (raw): {tupy_df.shape[0]}")

# Pega as colunas desejadas
columns_to_keep = ['text', 'hate']
tupy_df = tupy_df[columns_to_keep]
print(f"#\n# --- Colunas selecionadas: {columns_to_keep}")

# Padroniza os nomes das colunas
tupy_df.columns = ['text', 'is_toxic']
print(f"# --- Colunas renomeadas: {tupy_df.columns}")

# Normalizar a coluna 'is_toxic'
tupy_df['is_toxic'] = tupy_df['is_toxic'].map({'1':True, '0':False})
print(f"# --- Coluna 'is_toxic' normalizada para False (0) ou True (1)")

# Retira menções ao longo do texto, se antes do @ não for uma letra/palavra, e substitui por @user
tupy_df['text'] = tupy_df['text'].str.replace(r'(?<!\w)@\w+', '@user', regex=True)
tupy_df['text'] = tupy_df['text'].str.replace(r'\bUSER\b', '@user', regex=True)
print(f"# --- Menções normalizadas para '@user'")

# Retira URLs ao longo do texto
tupy_df['text'] = tupy_df['text'].str.replace(r'(?:https?://|www\.)\S+', '<url>', regex=True)
tupy_df['text'] = tupy_df['text'].str.replace(r'(?i)\blink\b(?=(?:\s*link\b)*\s*$)', '<url>', regex=True)
tupy_df['text'] = tupy_df['text'].str.replace(r'\bURL\b', '<url>', regex=True)
print(f"# --- Links normalizados para '<url>'")

# Normaliza os retweets, transformando para <RT>
tupy_df['text'] = tupy_df['text'].str.replace(r'\bRT\b', '<RT>', regex=True)
tupy_df['text'] = tupy_df['text'].str.replace(r'\brt\b', '<RT>', regex=True)
print(f"# --- Retweets normalizados para '<RT>'")


# Retira '\n' e substitui por ' '
tupy_df['text'] = tupy_df['text'].str.replace('\n', ' ', regex=False)
print(f"# --- \'\\n\' normalizado para ' '")

# Desconsidera todos os is_toxic sem valor (None)
tupy_df = tupy_df[tupy_df['is_toxic'].notna( )]
print(f"#\n# --- Registros com is_toxic=None removidos")
print(f"# Quantidade de registros após remoção de valores incompletos: {tupy_df.shape[0]}")

# Desconsidera entradas duplicadas
tupy_df = tupy_df.drop_duplicates(subset='text', keep='first')
print(f"#\n# --- Registros duplicados removidos")
print(f"# Quantidade de registros após remoção de duplicatas: {tupy_df.shape[0]}")

# Salva o DataFrame final como CSV
tupy_output = './data/02-cleaned/tupy.csv'
tupy_df.to_csv(tupy_output, index=False, quoting=csv.QUOTE_NONNUMERIC)
print(f"#\n# Salvo em: {tupy_output}")


##### LIMPEZA DE DADOS: TuPy (versão not-expanded)
# Número de registros (raw): 10000
#
# --- Colunas selecionadas: ['text', 'hate']
# --- Colunas renomeadas: Index(['text', 'is_toxic'], dtype='object')
# --- Coluna 'is_toxic' normalizada para False (0) ou True (1)
# --- Menções normalizadas para '@user'
# --- Links normalizados para '<url>'
# --- Retweets normalizados para '<RT>'
# --- '\n' normalizado para ' '
#
# --- Registros com is_toxic=None removidos
# Quantidade de registros após remoção de valores incompletos: 0
#
# --- Registros duplicados removidos
# Quantidade de registros após remoção de duplicatas: 0
#
# Salvo em: ./data/02-cleaned/tupy.csv
